In [111]:
import pandas as pd
import os
import glob
import numpy as np

In [112]:
path = os.getcwd()
csv_files = glob.glob(os.path.join(path, "..", "HR Data", "*.csv"))

In [113]:
hr_list = [pd.read_csv(f).drop(["Start time", "Date", "Duration", "Average heart rate (bpm)", "Calories"], axis=1) for f
           in csv_files]

for df in hr_list:
    df["Player Number"] = int(df["Player Number"][0])

hr_dataframe = pd.concat(hr_list)

In [114]:
def hr_stats(hri):
    return pd.Series({"player": hri["Player Number"].min(),
                         "hr_min": hri["HR (bpm)"].min(),
                         "hr_max": hri["HR (bpm)"].max(),
                         "hr_mean": hri["HR (bpm)"].mean(),
                         "hr_std": hri["HR (bpm)"].std(),
                         "hr_peak": hri["HR (bpm)"][hri["HR (bpm)"] >= 0.9 * hri["HR (bpm)"].max()].count() / hri[
                             "HR (bpm)"].count(),
                         "hr_bottom": hri["HR (bpm)"][hri["HR (bpm)"] <= 1.1 * hri["HR (bpm)"].min()].count() / hri[
                             "HR (bpm)"].count()})


In [115]:
hr_player_summary = hr_dataframe.groupby(["Player Number"]).apply(hr_stats)
hr_player_summary["player"] = hr_player_summary["player"].astype(np.int64)
hr_player_summary.sort_values(by="Player Number").to_csv(os.path.join(path, "..", "Processed", "hr_summary.csv"),index=False)

In [116]:
runs = pd.read_csv(os.path.join(path, "..", "Processed", "run_stats.csv"))

def calculate_run(hri,runs):


,player,attempt,run_start,run_end,time,chambers,boss_encounters
0,1,1,01:50:100000,06:30:500000,280.4,3,0
1,1,2,08:32:600000,12:52:200000,259.6,5,0
2,1,3,16:22:400000,27:00:100000,637.7,10,0
3,1,4,28:25:700000,42:55:200000,869.5,14,1
4,2,1,02:23:400000,06:44:700000,261.3,3,0
...,...,...,...,...,...,...,...
89,20,3,12:01:800000,16:16:800000,255.0,6,0
90,20,4,17:25:700000,19:50:500000,144.8,4,0
91,20,5,20:30:600000,22:38:500000,127.9,3,0
92,20,6,23:42:400000,29:15:500000,333.1,7,0
